## Functions trash bin

### Modifying file

In [1]:
import json

def get_data(filename='data.json'):
    with open(filename, 'r') as f:
        return json.load(f)


def save_data(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

### Filtering data

In [2]:
def filter_data(buffer=[], first_target=[], second_target=[], latest = []):
    data = get_data()
    result = dict()

    for k, v in data.items():
        if buffer and v['buffer'] not in buffer:
            continue
        if first_target and v['targets'].split()[0] not in first_target:
            continue
        if 


SyntaxError: invalid syntax (3918700285.py, line 10)

### Updating algs

In [5]:
VALID_CHARS = " UDFBRLMESudfbrlw'/:,2xyz"

def get_sheet_content(filename):
    with open(filename) as f:
        return [i.strip().split(';') for i in f]


def grid_to_alg_list(grid):
    result = []
    for i in range(1, len(grid)):
        for j in range(1, len(grid[0])):
            if grid[i][j]:
                alg = clean_alg_entry(grid[i][j])
                key = ";".join([grid[0][0], grid[0][j], grid[i][0], alg])
                result.append(key)
    return result


def clean_alg_entry(alg):
    # filtering out invalid chars
    alg = ''.join([i for i in alg if i in VALID_CHARS])

    # cleaning multiple spaces 
    alg = ' '.join(alg.split())

    # cleaning whitespaces before , : '
    i = 0
    while i < len(alg):
        if alg[i] in "':," and i > 0 and alg[i - 1] == " ":
            alg = alg[:i-1] + alg[i:]
        else:
            i += 1

    return alg


def only_latest_algs(data):
    return {k: v for k, v in data.items() if v['latest_alg']}


def keys_with_given_buffer_and_targets(data, buffer, first_target, second_target):
    return [k for k in data if [buffer, first_target, second_target] == k.split(';')[:3]]


def is_key_in_dict(data, key):
    return key in data


def new_record_from_key(key):
    key = key.split(';')

    return {
        'buffer': key[0],
        'first_target': key[1],
        'second_target': key[2],
        'alg': key[3],
        'results': [],
        'latest': True
    }


def update_algs():
    data = get_data()
    algs = get_sheet_content('algs.csv')
    algs_list = grid_to_alg_list(algs)

    for alg in algs_list:
        decoded_key = alg.split(';')
        existing_algs = keys_with_given_buffer_and_targets(data, decoded_key[0], decoded_key[1], decoded_key[2])

        for k in existing_algs:
            data[k]['latest'] = False

        if alg in data:
            data[alg]['latest'] = True
        else:
            data[alg] = new_record_from_key(alg)

    return data

In [6]:
data = update_algs()
save_data(data, 'data_updated.json') 

### Updating memo

In [14]:
def get_words_dict(grid):
    result = dict()
    for i in range(1, len(grid)):
        for j in range(1, len(grid[0])):
            if grid[i][j]:
                key = f'{grid[0][j]};{grid[i][0]}'
                result[key] = grid[i][j]
    return result


def update_words():
    data = get_data()
    words = get_sheet_content('memo.csv')
    words_dict = get_words_dict(words)

    for k, v in data.items():
        try:
            targets = f"{v['first_target']};{v['second_target']}"
            data[k]['word'] = words_dict[targets]
        if  in words_dict:
            

    return data




In [15]:
update_words()

KeyError: 'targets'

### Updating LPs

In [10]:
def get_lps_dict(grid):
    return {i[0]: i[1] for i in grid}

def update_LPs():
    data = get_data()
    lps = get_sheet_content('LP.csv')
    lps_dict = get_lps_dict(lps)

    for k, v in data.items():
        try:
            targets = v['targets'].split()
            data[k]['lp'] = ''.join([lps_dict[i] for i in targets])
        except KeyError:
            pass
    
    return data

In [12]:
update_LPs()

{"UF;UL;UB;U' M2 U: U2, M": {'buffer': 'UF',
  'first_target': 'UL',
  'second_target': 'UB',
  'alg': "U' M2 U: U2, M",
  'results': [],
  'latest': True},
 'UF;UR;UB;R2 U: S, R2': {'buffer': 'UF',
  'first_target': 'UR',
  'second_target': 'UB',
  'alg': 'R2 U: S, R2',
  'results': [],
  'latest': True},
 "UF;UB;UL;U' M2 U: M, U2": {'buffer': 'UF',
  'first_target': 'UB',
  'second_target': 'UL',
  'alg': "U' M2 U: M, U2",
  'results': [],
  'latest': True},
 'UF;UR;UL;M2 U: M, U2': {'buffer': 'UF',
  'first_target': 'UR',
  'second_target': 'UL',
  'alg': 'M2 U: M, U2',
  'results': [],
  'latest': True},
 "UF;UB;UR;R2 U': S, R2": {'buffer': 'UF',
  'first_target': 'UB',
  'second_target': 'UR',
  'alg': "R2 U': S, R2",
  'results': [],
  'latest': True},
 "UF;UL;UR;M2 U': M, U2": {'buffer': 'UF',
  'first_target': 'UL',
  'second_target': 'UR',
  'alg': "M2 U': M, U2",
  'results': [],
  'latest': True}}

### Running game

In [69]:
from random import shuffle
from datetime import datetime

def get_n_random_keys(data, n):
    data = {k: v for k, v in data.items() if v['latest']}
    keys = list(data)
    shuffle(keys)
    return keys[:n]    


def measure_time():
    start = datetime.now()
    input()
    end = datetime.now()
    delta = (end - start).total_seconds()
    return round(delta, 2)


def display_record(alg):
    if 'word' in alg:
        print(alg['word'])
    elif 'lp' in alg:
        print(alg['lp'])
    else:
        print(alg['targets'])


def get_response():
    print()
    response = input('Type Y to save results')
    return response == 'Y'


def add_entries(data, results):
    for k, v in results.items():
        data[k]['results'].append(v)

    save_data(data)

def run_game(n):
    results_dict = dict()
    data = get_data()
    algs = get_n_random_keys(data, n)
    for alg in algs:
        display_record(data[alg])
        exec_time = measure_time()
        results_dict[alg] = exec_time

    print('Results to be saved:')
    for k, v in results_dict.items():
        print(k.split(';')[1], v)

    if get_response():
        add_entries(data, results_dict)

### El śmietniko

In [43]:
alg = "[Ra    :  R  (1E ' 6  R    '&&   , %9 U       ']"
clean_alg_entry(alg)

"R: R E' R', U'"

In [54]:
words = get_sheet_content('memo.csv')
get_words_dict(words)

{'UL UB': 'Bat',
 'UR UB': 'Czas',
 'UB UL': 'Arbuz',
 'UR UL': 'Cable',
 'UB UR': 'Acid',
 'UL UR': 'Baca'}

In [60]:
run_game(5)

Arbuz
Czas
Baca
Cable
Acid
Results to be saved:
UB UL 1.32
UR UB 1.37
UL UR 1.19
UR UL 1.04
UB UR 1.17

